**Welcome** to the MLB Analytics Dashboard. This notebook is a data-driven exploration which brings together multiple sources of baseball data to discover insights into player performance, salary efficiency, and injury trends over time. By integrating WAR, financial data, and injury reports with player demographic information, we can get a holistic view of the modern game of baseball while learning a little (or hopefully a lot) about data science.

## Project Overview:
**Data Integration:**
- We will merge diverse datasets, including:
  - *Batting and Pitching Statistics:* season-level performance data
  - *Salary Information*: detailed financial data on player contracts
  - *Advanced Metrics (WAR):* our primary quantitative measure of player perfomance
  - *Demographic Data:* player names and birth years fron the Lahman database
  - *Injury Reports:* injury data that reveal patterns in player health year over year

**Analytical Motivations:**
1. *Pitch Percentage Mix Analysis:*
   - Explore through basic interactive charting through single year pitch type selection or trends over time
2. *Injury Analysis by Age and Position:*
   - Analyze injury trends using interactive visualizations like boxplots, violin plots, time series line plots, and grouped bar charts
   - Understand how injuries differ between pitchers and position players and how age influences types and frequency of injury eventes
   - Investigate monthly trends in injuries to expose seasonal patterns and long term shifts
3. *War vs Salary Analysis:*
   - Explore translation of salary investments into on-field performance
   - Spotlight both players who demonstrate exceptional value and those who demonstrate the sinkhole their contracts have become on their teams









### Installing the Required Code Packages
**If you are running this environment online for the first time:**
1. You will need to run some `pip install...` commands in order to to run the code correctly
2. They will enable your use of pandas for data manipulation, plotly for making interactive charts, ipywidgets for additional interactivity, seaborn for making static charts, and openpyxl to deal with Excel files

Run the following cell once so you have access to all the required packages to run this project

## Visualization Library Choices:

#### Seaborn
Seaborn is a Python data visualization library that runs on Matplotlib. It is open source and freely available on GitHub. You can install it easily in the notebook using pip:
`!pip install seaborn`

**Why Seaborn?**
- *Ease of Use:* Simplifies creation of appealing statistical graphics with less coding
- *Integration:* Works seamlessly in Jupyter notebook environment
- *Declarative Style:* Uses mostly a declarative approach which allows users to specify what they want to see without having to conduct extensive procedural programming

**Why Not Seaborn?**
- It's less customizable than lower-level libraries like Matplotlib
- Works best for static visualizations because of limited funcationality with interactivity

#### Plotly
Plotly is an interactive visualization library for Python, and it's an open source project. You can install it in the notebook using pip just like Seaborn:
`!pip install plotly`

**Why Plotly?**
- *Interactivity:* Enables zooming, hovering, and selecting data points dynamically
- *Web Integration:* Embeds directly into web apps which is ideal for interactive dashboards
- *Both Declarative and Procedural:* Offers both declarative methods with Plotly Express and procedural methods with Plotly Graph Objects
- *Jupyter Compatibility:* Integrates fully into Jupyter notebook environment, providing interactive visual feedback immediately

**Why Not Plotly?**
- A little more intricate to code with, making it steeper of a learning curve than working with something like Seaborn
- May require additional resources to render interactive plots smoothly, especially with large sets of data

**Can We Use Both?**
- *We can and we will!* Using both of these libraries together allows us to leverage the strengths of each so our plots are even more effective and expressive
  - We will use Seaborn for exploratory static visualizations and plotly for the detailed, interactive, exploratory, and explanatory dashboards

In [ ]:
# Uncomment the following commands and run the cell if this is your first time running this environment
# !pip install pandas
# !pip install plotly
# !pip install ipywidgets
# !pip install seaborn

# MLB Pitch Usage Breakdown and Analysis: A Step by Step Tutorial

#### Objective: Basic Plot
- **Explore** how pitch usage in the MLB has evolved over time between 2015-2024
- **Build** an interactive chart using plotly to visualize the mixed percentage of pitch types thrown across seasons
- **Understand** season-level performance by incorporating StatCast metrics when hovering over a single season pitch percentage bar, or dot on the pitch-type/year/type-percentage time series plot

#### Data Source:
- `stats_pitchers_all.csv`: contains season level pitch count data for all pitch types accounted for during the StatCast era
  - We're focusing on completed seasons, so 2015-2024 as of the writing of this tutorial

#### What We Will Learn Making Our Plot:
**How To:**
1. Load and filter data from datasets
2. Transform the data into a usable format - we will need our data in the long format for this chart
3. Calculate the percentages of each pitch type thrown per season
4. Create an interactive chart using plotly and ipywidgets to enable the viewer to flip back and forth between single-season bar charts and multi-season line charts

#### Where We'll Go Afterward:
**Advance** our skills gained from making this chart to dive deeper into more intricate charts, using more data, to deliver more interesting insights

## Import the Required Libraries

#### We must import the following libraries in order to do our analysis:

In [ ]:
# Pandas - used for data manipulation
# Documentation: http://pandas.pydata.org/
import pandas as pd

# Plotly - interactive library we are going to use for plotting
# Documentation: http://plotly.com/python/plotly-express/
import plotly.graph_objects as go
import plotly.express as px

# ipywidgets - allows addition of interactivity with UI elements in the Jupyter notebook
# Documentation: http://ipywidgets.readthedocs.io/
import ipywidgets as widgets

# display - the function from the IPython library that allows us to render the widgets and plots described above
from IPython.display import display

### Checking notebook readiness for Plotly

Before we get too deep into our tutorial, let's make sure that Plotly is working in our environment. For this quick check, we're goign to use Plotly Express with the built-in Iris dataset to plot the most basic of scatter plots

In [ ]:
##### Before we get started, we can check one last thing.....is Plotly ready to use yet?
# We already imported plotly.express as px above
# The iris dataset is built-in so we plot a figure to know we're good to go
df = px.data.iris()

# Create scatter plot using the Iris dataset
# The x axis = 'sepal_width' and the y axis = 'sepal_length'
# Using 'px.scatter()' creates a scatter plot for us
fig = px.scatter(df, x='sepal_width', y='sepal_length')

# Display the figure
fig.show()

# If it works, we should see a basic scatter plot rendered with dots that have hoverover interactivity

**If you see** a scatter plot with interactive dots including hovering over any dots to see details, then Plotly is ready to use in our project

## Step 1: Loading the Pitching Data
- We will use our already-downloaded dataset, `stats_pitchers_all.csv`, which has the pitch count data we need
- We will focus on the years we're analyzing: 2015-2024

In [ ]:
# Load the pitching data from the dataset into a dataframe we'll call pitch_df
# This dataset contains all the pitching records we need
pitch_df = pd.read_csv('assets/stats_pitchers_all.csv')

# Take all the pitches loaded into the new df, and filter for 2015-2024
pitch_df = pitch_df[pitch_df['year'].between(2015, 2024)]

# What does our data look like?
# It's always a great idea to look at our data to see what we're working with
# Let's get a sanity check and print a few random records from the dataframe to check our data so far
print(f'A small sample of pitches within our specified year-range:\n {pitch_df.sample(5)}')


## Step 2: Data Transformation: Mapping and Melting
- We will use the melt function to convert our data from wide to long so we only have one column per type of pitch and one column for count
- We will also create a pitch type map for easier pitch labels and understanding for downtream data minipulation

#### In this step we will:
1. *Define our Mapping:* we are going to create a dictionary called `pitch_type_map` to convert the cryptic column names into a more readable format
   - Documentation: [Pandas Mapping Documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html)
2. *Melt the Data:* we are going to use the `pd.melt()` function to reshape the dataframe from wide to long which will result in two columns:
   - A pitch type column
   - A a column with pitch count for each pitch type
   - Documentation: [Pandas Melt Documentation](http://pandas.pydata.org/docs/reference/api/pandas.melt.html)
3. *Map the Pitch Type Names:* once we melt the data, we can use `pitch_type_map` to replace the ugly names with clean and readable ones
4. *Clean the Numeric Data:* we are goign to convert the pitch count values to a numeric type using `pd.to_numeric` and replace non-numeric values with zero
5. *Inspect the Result:* finally, we will print a small random sample of the transformed data to verify it passes the smell test

In [ ]:
##### Define a map from hard to understand column names so we can use easier ones
pitch_type_map = {
    'n_fastball_formatted': 'Fastball',
    'n_si_formatted': 'Sinker',
    'n_ch_formatted': 'Changeup',
    'n_kn_formatted': 'Knuckleball',
    'n_cu_formatted': 'Curveball',
    'n_sl_formatted': 'Slider'
}

# Convert from wide to long using the melt function
# You have to unpivot the dataframe in order to be able to perform 
# ...multiline-plotting by pitch type or grouped analysis
pitch_long_df = pitch_df.melt(
    id_vars=['year'],             # keep this column as is
    value_vars=list(pitch_df),    # unpivot pitch_df into 2 cols: pitch type and count
    var_name='raw_pitch_col',     # new column holds original col names
    value_name='raw_pitch_count'  # new column holds numeric count of each pitch type
)

# Map the original StatCast column names to new, clean, pitch labels
# - 'pitch_type' vals will be the clean vals used for charts, legends, and code
pitch_long_df['pitch_type'] = pitch_long_df['raw_pitch_col'].map(pitch_type_map)

# Enforce all pitch counts to be numeric and cleaned
# - pd.to_numeric('blah blah', errors='coerce') converts all vals to numbers and sets non-convertible vals to NaN
# - prevents downstream data manipulation and math from being skewed or producing errors
# - 'fillna(0)' replaces any NaN with 0 for clean, numeric datasets
pitch_long_df['raw_pitch_count'] = pd.to_numeric(pitch_long_df['raw_pitch_count'], errors='coerce').fillna(0)
# Documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

# Let's see what it looks like:
# Print a random sample of 5 rows from the transformed dataframe
# We expect to see the columns 'year', 'raw_pitch_col', 'raw_pitch_count', and 'pitch_type'
print(f'A small of our transformed data:\n{pitch_long_df.sample(50)}')


## Step 3: Aggregating the Data - Total Pitch Usage and By Season
We are going to aggregate pitch data to understand how pitches are used and vary by season which allows for the critical part of our analysis: to determine trends over time

#### In this step we will:
1. *Group and Sum Pitch Counts:* we will group our now-long-format data by `year` and `pitch_type` and get their sums
   - Produces a dataframe we'll call `pitch_mix_df` that tells us how many pitches were thrown by season and pitch type
2. *Calculate Season Totals:* we will calculate the total number of pitches thrown each season by summing counts across all pitch types
   - Gets stored in a dataframe we'll call `season_totals_df` and name the summed column `total_pitches` for clarity
3. *Merge and Calculate Percentages:* we will merge the pitch mix dataframe with season totals so each row has a corresponding season with its pitch count
   - From this we calculate the percentage of each pitch type thrown per season
4. *Calculate Season Level StatCast Averages:* we will calculate the yearly averages for key stats like `whiff_percent`, `f_strike_percent`, `in_zone_percent`, and `groundballs_percent`
   - Documentation: [Pandas GroupBy.mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html)
5. *Merge Season Averages with Trend Data:* finally, we will merge the season level averages into our new trend dataframe `trend_df` and as per usual rename the columns for clarity
6. *Inspect the Result:* we will print a small sample of the final dataframe to check everything looks the way we think it's supposed to

In [ ]:
##### Aggregation of pitch usages by season
# Group by year and pitch type, and then sum every pitch type occurence by year
# Using the 'groupby()' method with the agg() function, we create a new column called 'pitch_count'
pitch_mix_df = pitch_long_df.groupby(['year', 'pitch_type'], as_index=False).agg(
    pitch_count=('raw_pitch_count', 'sum')   # This part sums up all pitch counts for each group
)

# Calculate total pitch count per year
# Will use this later when we calculate total percentage of pitch type thrown
# Group by the 'year' and sum the 'pitch_count' to get total pitches thrown each year
# Set 'pitch_count' to 'total_pitches' for ease of upcoming merge and math
season_totals_df = pitch_mix_df.groupby('year', as_index=False)['pitch_count'].sum().rename(
    columns={'pitch_count': 'total_pitches'}
)
# Documentation on 'sum()': http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html

# Merge the total pitches with pitch mix and get percentages
# - Join pitch type counts on total pitches per year
# - After that, we can tell the pitch types and counts per season
# We are merging two dataframes on the 'year' column
trend_df = pitch_mix_df.merge(season_totals_df, on='year')

# Calculate pitch percentage
# - Shows how much any pitch is thrown in any season
trend_df['pitch_pct'] = trend_df['pitch_count'] / trend_df['total_pitches'] * 100

# Calculate yearly averages of key StatCast stats:
# - 'whiff_percent': percentage of batter swings and misses
# - 'f_strike_percent': percentage of times the first pitch is a strike
# - 'in_zone_percent': how often a pitcher throws a strike
# - 'groundballs_percent': percentage of balls hit into play on the ground
# This data is going to be used for annotations and hover info in our plots
season_averages_df = pitch_df.groupby('year', as_index=False)[[
  'whiff_percent', 'f_strike_percent', 'in_zone_percent', 'groundballs_percent'
]].mean()

# Pull StatCast averages into our trend data by merging season averages into our 'trend_df'
# Will use this to show some useful stats when hovering, concise names -> easier to read
trend_df = trend_df.merge(season_averages_df, on='year').rename(columns={
    'whiff_percent': 'whiff_pct',
    'f_strike_percent': 'first_strike',
    'in_zone_percent': 'in_zone',
    'groundballs_percent': 'groundball_pct'
})

# Now let's take a look at what our aggregated and merged data look like by taking a look at the first 10 rows
print(f'Aggregated Pitch Mix Data With Percentages and Season-Level Stats:\n{trend_df.head(10)}')


**We now have** all the necessary data aggregated into the `trend_df` dataframe which we will use for our visualizations

## Step 4: Defining Our Visualization Settings
We are goign to set up some key variables that help control how our chart presents the different pitch types

#### In this step we will:
1. *Define the Pitch Type Order:* we will define the `pitch_type_order` list which defines the pitch types sequence for the x-axis, which is important for visualization consistency
   - In our case we want 'Fastball' to always be first, then 'Sinker', then 'Changeup', 'Knuckleball', 'Curveball', and finally 'Slider'
2. *Create our Pitch Color Map:* we will create a pitch color dictionary we'll call `pitch_color_map` which will map every pitch type to its own, specific color using hex codes
   - Ensures each pitch type can be picked out distinctly
   - Find the documentation on dictionaries here: [Python Data Structures - Dictionaries](http://docs.python.org/3/tutorial/datastructures.html#dictionaries)

***Note:*** general python data structures documentation can be found [here](http://docs.python.org/3/tutorial/datastructures.html)

In [ ]:
##### Define the order for the pitch types in the chart 
# The following list is the sequence the pitches appear on the chart along the x-axis
pitch_type_order = ['Fastball', 'Sinker', 'Changeup', 'Knuckleball', 'Curveball', 'Slider']

# Define a color map so each pitch type has its own color
# - Use the pitch types as keys, and color hex values as the chosen colors
pitch_color_map = {
    'Fastball': '#132448',     # Yankees Navy
    'Sinker': '#00A3E0',       # Marlins Teal
    'Changeup': '#EFB21E',     # Athletics Gold
    'Knuckleball': '#582C83',  # Rockies Purple
    'Curveball': '#2F241D',    # Padres Brown
    'Slider': '#BD3039'         # Red Sox Red
}
# You can find more documentation on dictionaries at:
# http://docs.python.org/3/tutorial/datastructures.html#dictionaries


## Step 5: Create Our Interactive Dropdown Widget
It's about that time we make our first widget; it will allow users to choose a season from a range. We'll be using ipywidgets to build user interface elements that lets the user choose between an individual season or 'All Years' from the dropdown menu 

#### In this step we will:
1. *Set up the Widget Options:*
   - The dropdown will have an 'All Years' option or individual season options between 2015 and 2024
   - Use list comprehension to generate a list of years as strings
2. ***Creating the Widget:***
   - Use `widgets.Dropdown()` to create our dropdown widget
   - Give the widget the default value of 'All Years' and create a descritive label for it
3. *Display the Widget:*
   - Use `display()` to show dropdown widget in our notebook, creating interactivity


In [ ]:
##### Creating the Interactive Dropdown Widget with ipywidgets
# Make the widget for selecting a season
# We use list comprehension here with '[str(year) for year in range(2015, 2025)]'
# - When combined with 'All Years' we get our full set of options we need
year_dropdown_widget = widgets.Dropdown(
    # 'All Years' and then the individual seasons
    options=['All Years'] + [str(year) for year in range(2015, 2025)],
    value='All Years',         # Default value is 'All Years'
    description='Year:'        # Label displayed next to widget
)

# Display the dropdown widget
display(year_dropdown_widget)


#### If everything was done right, up to this point, you should see a dropdown menu directly above.

The default value should be set to "All Years", with the years 2015 - 2024 displayed immediately afterward, underneath, when the dropdown is clicked

## Step 6: Main Chart Function: Plotting MLB Pitch Percentage Mix Trends
We are going to define our main chart function and then we'll have our first chart visualization done! We'll call our function `plot_pitch_mix_chart()` 

#### In this step we will:
1. *Create a Base Figure:* we will create an empty Plotly figure to be our canvas for our plot
2. *Define our Hover Templates:* we are going to build two templates, one for the line chart and one for the bar charts, by using Plotly's `%{variable}` syntax
   - Templates will include detailed information such as pitch mix percentage and interesting season-level StatCast stats for that year
   - Documentation: [Plotly Hover Template](https://plotly.com/python/customizing-hover-text/)
3. *Logically Handle Two Visualizations:*
   - **All Years View (Multi-Year Line Chart):**
     - When selected, each pitch type gets its own line trace in the defined order and `pitch_color_map` for consistent styling
     - Add update menu buttons that lets the user 'Select All' or 'Deselect All' traces
   - **Single Year View (Bar Chart):**
     - When an individual year is selected we make then create a bar chart to visualize the pitch mix percentage for that year
     - Bars are ordered in the previously defined `pitch_type_order` so the bars appear in the same place with the same colors for every year
4. *Update the Figure's Layout:* we will update the layout by creating or changing the titles, axis labels, margins, and a creating an interactive legend
   - Add annotations when instructions for the user
5. *Display and Analyze the Interactive Output:* by using `fig.show()` we can view our first interactive chart!


In [ ]:
##### This is it, our main chart function. This is where we take our data and turn it into an 'All Years' 
#    - trends in pitch mixes, with added seasonal snapshots of those percentages
# This function handles all the visualization logic for the plot
# Season-level statistics will be enabled upon hovering on both single season and 'all years' charts
def plot_pitch_mix_chart(selected_year):
    # Create empty plotly figure to begin the chart-building
    fig = go.Figure()

    # Create Hover Template:
    # - Will use Plotly's %{variable} syntax for hoverover tooltips
    # -  Each chart shows pitch mix pct and StatCast stats fed into chart
    # 'hover_lines' is used for the multiyear line chart:
    hover_lines = (
        "<b>%{fullData.name}</b><br>Year: %{x}<br>% Mix: %{y:.1f}%<br>"
        "<i>Overall Year Stats:</i><br>"
        "Whiff%: %{customdata[0]:.1f}% | 1st-Strike%: %{customdata[1]:.1f}%<br>"
        "In-Zone%: %{customdata[2]:.1f}% | Groundball%: %{customdata[3]:.1f}%<extra></extra>"
    )
    # 'hover_bars' used for the single year bar-charts:
    hover_bars = (
        "<b>%{x}</b><br>% Mix: %{y:.1f}%<br>"
        "<i>Overall Year Stats:</i><br>"
        "Whiff%: %{customdata[0]:.1f}% | 1st-Strike%: %{customdata[1]:.1f}%<br>"
        "In-Zone%:%{customdata[2]:.1f}% | Groundball%: %{customdata[3]:.1f}%<extra></extra>"
    )

    # Handle Both Visualization Modes
    # First up is the "All Years" multiyear line chart where each pitch type gets its own line
    # Each line uses pitch_color_map and adds hoverover capabilities
    # Check if "All Years" is selected
    if selected_year == 'All Years':
        # For each pitch type, add a line trace for multiyear trends
        for pitch in pitch_type_order:
            # Filter aggregated data for selected pitch type
            df_pitch = trend_df[trend_df['pitch_type'] == pitch]
            # Add trace to the figure with lines and markers
            fig.add_trace(go.Scatter(
                x=df_pitch['year'],           # x-axis: years
                y=df_pitch['pitch_pct'],      # y-axis: pitch mix %
                name=pitch,                   # trace name for legend  
                mode='lines+markers',         # use both lines and markers
                line=dict(color=pitch_color_map[pitch]),   # use our colors
                customdata=df_pitch[[         # add season stats for hoverover info
                    'whiff_pct',
                    'first_strike',
                    'in_zone',
                    'groundball_pct']],
                hovertemplate=hover_lines     # use hover template we built above
            ))

            # Add select/deselect buttons for interactivity
            updatemenus = [dict(
                type='buttons',
                direction='down',
                x=1.02, y=0.65,
                xanchor='left', yanchor='top',
                showactive=False,
                buttons=[
                    dict(label='Select All', method='update', args=[{'visible': [True]*6}, {}]),
                    dict(label='Deselect All', method='update', args=[{'visible': ['legendonly']*6}, {}])
                ]
            )]
            annotation_note = 'Click legend items or use buttons to filter pitches. Hover for overall season stats.'
            plot_title = 'Pitch Mix Percentage Trends Between 2015-2024'
    else:
        # Single year selected: then bar chart
        # For specific year, convert to integer
        yr = int(selected_year)

        # Reorder data for that year using pitch_type_order
        df_year = trend_df[trend_df['year'] == yr].set_index('pitch_type').reindex(pitch_type_order).reset_index()

        # Add bar trace for selected year
        fig.add_trace(go.Bar(
            x=df_year['pitch_type'],                                 # x-axis: pitch types
            y=df_year['pitch_pct'],                        # y-axis: pitch mix percentages
            marker_color=[pitch_color_map[p] for p in pitch_type_order],      # our colors
            customdata=df_year[[ 
                'whiff_pct',
                'first_strike',
                'in_zone',
                'groundball_pct'
            ]].values,
            text=df_year['pitch_pct'].round(1).astype(str) + '%',    # display pct on bars
            textposition='auto',                         # automatically position the text
            hovertemplate=hover_bars                         # use our hover_bars template
        ))

        updatemenus = []    # no updatemenus for single year view
        annotation_note = 'Hover for overall season stats'
        plot_title = f'Pitch Mix Percentage - {yr}'

    # Update Final Figure Layout and UI Styling
    fig.update_layout(
        title=dict(text=plot_title, x=0.02, xanchor='left', font=dict(size=24)),
        xaxis=dict(tickmode='linear', dtick=1),       # show an x-tick for every season
        yaxis=dict(title=dict(text='Percentage (%)', font=dict(size=14)), range=[-5, None]),
        legend=dict(x=1.02, y=1),
        updatemenus=updatemenus,
        template='plotly_white',
        height=650,
        margin=dict(t=100, b=60, l=50, r=200)
    )

    # Add annotation with instructions above the chart
    fig.add_annotation(
        text=annotation_note,
        xref='paper', yref='paper',
        x=0.02, y=1.03,
        showarrow=False,
        font=dict(size=12)
    )

    # Display the interactive chart
    fig.show()

# Connect the Widget to the Plot
# Link the dropdown widget to out plot function with 'interactive_output'
plot_output = widgets.interactive_output(plot_pitch_mix_chart, {'selected_year': year_dropdown_widget})

# Stack vertically and display the widget and plot together
display(widgets.VBox([year_dropdown_widget, plot_output]))


## What did our code just do?

**Defined** our plotting function, `plot_pitch_mix_chart()`, which creates our chart based on the season selected:
- *Multi-Year View*: when the season chosen is "All Years", each pitch type is looped over for each year which creates a line trace. Each of those now contains detailed information which is displayed upon hovering over
- *Single-Year View*: when the season chosen is a specific year, a bar chart with pitch percentages is created for that season, also with detailed information upon hover
- Connected season selection widget to the plotting function
- Created interactive update menus in the multi-year view to toggle visibility for specific pitches on and off in the chart
  - This is dynamically updated as users select different seasons

# Conclusion: MLB Pitch Usage Breakdown

**Great Job!**
You just completed your first MLB data analysis project by looking into how pitchers use their arsenal both over the course of a season and over time. Over the course of this analysis we:
1. Loaded and filtered our pitching data to focus on modern StatCast data
2. Transformed our data from wide to long format using `pd.melt()` and mapped unintuitive column names to easily understandable labels
3. Aggregated the pitching data to calculate pitching mix percentages, and merged in season-level StatCast metrics for hoverover data
4. Defined order and color mapping so Plotly interactivity was consistent and effective
5. Built an interactive chart using ipywidgets so we can change between single and "All Years", hoverover tooltips, and legend interactivity allowing the user to view any combination of pitches they want

**Don't get too comfortable:** 
Now that you've warmed up a little bit, we're going to put your newly developed skills to the test. We're moving up to Double-A, and taking on a project where we can view the interplay of age, position category, and injuries with MLB players. 

**What are we going to try to find out?**
1. How does age factor into MLB injury trends?
2. What are the distributions of injuries across age, position groups, and injury categories and how can static libraries like seaborn and interactive libraries like plotly help us understand that data?
3. What can we learn about general player longevity and health?

Using what we learned in our basic pitch mix analysis, we are going to step up our game to try to get a deeper understanding of the many dynamics affecting the game of baseball

# MLB Injury Analysis by Age and Position Group: A Step by Step Tutorial

#### Objective: Intermediate Plot
1. **Investigate** the relationship between player age and injury events in the MLB for seasons between 2020-2024
2. **Merge** injury reports from multiple seasons with player demographic data to calculate age of injuries
3. **Clean** and simplify position group data into two groups: 'Pitcher' and 'Position Player'
4. **Categorize** the different injury types into intuitive groups for manageability of effectiveness and expressiveness of visualization
   - We will be using groups such as the following: elbow, illness, leg muscle, etc.
5. **Visualize** data distributions and trends using both static (Seaborn) and interactive (Plotly) visualization libraries

#### Data Sources:
1. **Injury Data:** individual season Fangraphs reports from 2020-2024, stored as Excel files
2. **People Data:** uses `people.csv' from Lahman for player names and birth years for calculations of ages at time of injury

#### What We Will Learn:
**How To:**
1. Load and combine injury reports from multiple MLB seasons
2. Import and process demographic player data to get their ages
3. Merge multiple datasets to calculate players' ages at time of injury
4. Clean and categorize players into two position groups: 'Pitcher' and 'Position Player'
5. Write custom function to group and categorize the different injuries into a manageable number of groups
6. Visualize injury data using the static and interactive charts

## Import the Required Libraries

#### We must import the following libraries in order to do our analysis:
Though we've done some of these above, we'll put in all the ones we need to use here for instructive purposes, even if it's a repeat import

In [ ]:
# Pandas - used for data manipulation
# Documentation: http://pandas.pydata.org/
import pandas as pd

# Seaborn - used for static visualizations
# Builds on Matplotlib for easier generation of statistical plots
# Documentation: http://seaborn.pydata.org/
import seaborn as sns

# Matplotlib - foundation for many Python visualizations; used for plotting
# Documentation: http://matplotlib.org/
import matplotlib.pyplot as plt
# - Create custom legend entries
# Documentation: http://matplotlib.org/stable/api/patches_api.html
from matplotlib.patches import Patch
# - Enables precise control over ticks and labels
# Documentation: http://matplotlib.org/stable/api/ticker_api.html
from matplotlib.ticker import FixedLocator, FixedFormatter

# Plotly - interactive library we are going to use for dynamic plotting
# Documentation: http://plotly.com/python/plotly-express/
# graph_objects is used for detailed plot construction
import plotly.graph_objects as go
# Used for quick and easy plots
import plotly.express as px

# re - used for regular expressions and text cleaning
# Documentation: http://docs.python.org/3/library/re.html
import re 

# ipywidgets - allows addition of interactivity with UI elements in the Jupyter notebook
# Documentation: http://ipywidgets.readthedocs.io/
import ipywidgets as widgets

# display - the function from the IPython library that allows us to render the widgets and plots described above
from IPython.display import display

## Step 1: Load and Combine Injury Data
We are going to load injury reports from multiple seasons (2020-2024) and combining them into a single data frame so we can analyze MLB injury trends over multiple seasons

#### In this step we will:
1. *Define Our Seasons of Interest:* by creating a list of years (2020-2024) we want to include in our study
2. *Load Each Season's Data:* construct the file path to the corresponding Excel file (something like `fangraphs_injury_report_2020.xlsx`)
   - Use `pd.read_excel()` to load injury data to a dataframe
   - Add new column we'll call `season` to show which season a record belongs to
   - Store each seasons dataframe in a list we'll call `injury_dfs`
3. *Combine All DataFrames:* by merging the individual season dataframes with `pd.concat()` into a single dataframe called `injuries`
   - Print shape of combined dataframe for proof the data was successfully loaded and merged


In [ ]:
##### Load and combine the injury reports
# Limit the seasons we're interested in using for the study
years = [2020, 2021, 2022, 2023, 2024]

# Initialize a list to hold each season's dataframe
injury_dfs = []

# Loop over each year to get corresponding injury report, and load the data
for year in years:
    # construct filepath
    file_path=f'assets/fangraphs_injury_report_{year}.xlsx'
    # this is for debugging: we can track progress if a file doesn't load
    print(f'Loading injury data for {year} from {file_path}...')
    # Use Pandas to load the excel file into the dataframe
    # Documentation: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
    df_year = pd.read_excel(file_path)
    # each row -> corresponding season
    df_year['season'] = year
    # append to list
    injury_dfs.append(df_year)

# Concatenate all dataframes into one
# 'ignore_index=True' resets index in new dataframe so there's no break in data
# Documentation: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
injuries = pd.concat(injury_dfs, ignore_index=True)

# Print the shape of combined dataframe to verify if merge was successful
print(f'Combined Injuries Data Shape: {injuries.shape}')


## Step 2: Load People Data and Merge with Injuries to Get Birth Years
We are going to load the player demographic data from `people.csv` and merge it with out injury data. The intent is to end up with the `birthyear` information for each player, which we'll need to calculate age at time of injury

#### For this step we're going to:
1. *Load the People Data:* by using `pd.read_csv()` to load the data from `people.csv`, making sure to specify encoding='latin1' so we don't get errors trying to read the file
   - Then print the shape of the dataframe to verify data is loaded correctly
2. *Standardize Column Names:* strip whitespace and convert column names to lowercase for both injury and people dataframes
    - When we merge the dataframes, this part helps prevent mismatches
3. *Standardize Player Names:* create a new column we'll call `player_name` in the injury dataframe by cleaning the `name` column
   - Convert to string, strip the whitespace, convert to lowercase, and remove periods and commas
   - In the people dataframe we will combine the `namefirst` and `namelast` columns after we clean them to derive our new `player_name` column
   - Documentation on Pandas String Methods: [Pandas String Methods](https://pandas.pydata.org/docs/user_guide/text.html)
4. *Merge DataFrames:* now we're going to merge the dataframes on the `player_name` column they both have and it's going to be a left join so all the injury records will be kept
   - After the merge, we are going to filter out the rows where there is no `birthyear` so our ages are legitimate
     - Upon closer inspection of raw data, there's multiple people in the set over 160 years old so there may be some artifact that needs exploring. Check it out if you're curious!
5. *Verify Merge Success:* print the shape of the merged dataframe to confirm our expected outcome

In [ ]:
##### Load the people data and merge to get the 'birthyear'
# Load demographic data from 'people.csv'
# Use `pd.read_csv()` with encoding='latin1' in case there are special characters
people_df = pd.read_csv('assets/people.csv', encoding='latin1')
print(f'People datashape: {people_df.shape}')

# Standardize column names
# Strip whitespace and convert to lower-case
injuries.columns = injuries.columns.str.strip().str.lower()
people_df.columns = people_df.columns.str.strip().str.lower()

# Create our standardized 'player_name' column in the injuries df
# Remove dots and commas and convert to lowercase for consistency
injuries['player_name'] = (injuries['name'].astype(str)
                           .str.strip()
                           .str.lower()
                           .str.replace('.', '', regex=False)
                           .str.replace(',', '', regex=False))

# for the people dataframe comebine first and last names to get and create 'player_name' column
people_df['player_name'] = ((people_df['namefirst'].astype(str).str.strip() + ' ' +
                            people_df['namelast'].astype(str).str.strip())
                            .str.lower()
                            .str.replace('.', '', regex=False)
                            .str.replace(',', '', regex=False))

# Merge injuries and people dataframes on `player_name` to bring in birth year
# Left join keeps all injury records, even if there's no match in people_df
# Documentation: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html
df = pd.merge(injuries, people_df[['player_name', 'birthyear']], on='player_name', how='left')

# Filter out rows where birthyear is missing for valid age calculations
df = df[df['birthyear'].notna()].copy()

# Confirm merge and filtering are successful
print(f'Data shape after merge: {df.shape}')


## Step 3: Calculate Age at Injury
How old were the players when they got injured? We're going to compute that during this step. This calculation is essential for our study because it allows us to correlate injuries with age to better understand the potential vulnerabilities of certain position groups, to suffer certain types of injuries, at certain ages

#### For this step we're going to:
1. *Standardize the 'year' Column:* by converting the `season` column to a numeric type and store it in the new column we'll call `year`
   - This will ensure consistency across our data
   - Documentation: [Pandas 'to_numeric' Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html)
2. *Convert Injury Date to Datetime:* by converting the `injury / surgery date` column into a datetime object
   - Enables us to extract injury year easier
   - Documentation: [Pandas 'to_datetime' Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
3. *Calculate Age at Injury:* by subtracting the player's birth year from year extracted from the player's injury date
4. *Filter for Realistic Ages:* be dropping any records where the calculated age is less than or equal to 16 or greater than or equal to 49
   - Makes sure that only plausible ages remain for analysis
5. *Sanity Check:* we're going to inspect our dataframe to ensure our new column we'll call `age_at_injury` looks like we'd expect it to look


In [ ]:
##### Calculate player ages at their time of injury
# Convert 'season' column to numeric and store in new 'year' column for consistency
# The errors='coerce' parameter converts invalid parsing situations into NaN, allowing only clean numeric data
# Standardizing 'season' as numeric maintains concistency throughout and allows for numeric comparisons
df['year'] = pd.to_numeric(df['season'], errors='coerce')

# Convert 'injury / surgery date' to datetime object
# Specify format: '%m/%d/%y' for correct parsing
# Documentation: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
df['injury_date'] = pd.to_datetime(df['injury / surgery date'], format='%m/%d/%y', errors='coerce')

# Calculate age at injury as difference between injury year and birth year
# Here we extract th eyear from the `injury_date` using .dt.year
df['age_at_injury'] = df['injury_date'].dt.year - df['birthyear']

# Filter data to include only realistic age ranges
# Keep only rows where `age_at_injury` > 16 and <= 49
df = df[(df['age_at_injury'] > 16) & (df['age_at_injury'] <= 49)]

# Sanity check: what does our dataframe look like? Does it look like you would expect at this point?
df


## Step 4: Simplify Position Groups and Map Injury Categories
We are going to simplify the player position data and map injury descriptions into our defined categories 

#### For this step we're going to:
1. *Simplify Position Groups:* by classifying the players into two groups: 'Pitcher' and 'Position Player'
   - Check player `pos` value against a set of labels that indicate pitching roles like 'SP' for starting pitcher or 'RP' for relief pitcher
     - All non-pitchers are 'Position Players'
     - Helps for consistency in grouping downstream
2. *Categorize Injuries:* by writing a custom function we'll call `categorize_injury()` which looks at injury description text and places it into a standardized category
   - Function looks for key words in the text like 'shoulder', 'elbow', 'back', etc., and assigns a defined category like 'Elbow Injury' or 'Back Injury'
   - Helps us group injuries broad enough for easy visualizing our areas of interest, but granular enough for meaningful analysis
3. *Define Order and Clean Labels:* by creating an ordered list we'll call `injury_order` to forcibly control the display order in charts and reports
   - At the same time, we'll create a list of clean labels by removing the word 'Injury' from category names for concision and user-friendliness

In [ ]:
# Define a list of position labels which identify if player is a pitcher
# If player 'pos' in this list, then they are a 'Pitcher'
pitcher_labels = ['SP', 'RP', 'SP/RP', 'RP/SP']

# Simplify the position groups by creating new column 'position_group'
# If player 'pos' value is one of the 'pitcher_labels' they're 'pitcher', otherwise 'position_player'
# - Here we use '.apply()' to check each 'pos' value and lambda function returns 'Pitcher' for any pitcher_labels
df['position_group'] = df['pos'].apply(lambda x: 'Pitcher' if x in pitcher_labels else 'Position Player')


# Map injuries to categories using our custom function
# Our use of 'any()' and list comprehensions make our checks efficient
# Documentation on string methods: http://pandas.pydata.org/docs/user_guide/text.html
def categorize_injury(injury_text):
    # Convert injury text to lowercase
    itext = str(injury_text).lower()
    # Check text for keywords to figure out which category we put the injuries into
    if any(w in itext for w in ['elbow', 'tommy john']):
        return 'Elbow Injury'
    elif any(w in itext for w in ['shoulder']):
        return 'Shoulder Injury'
    elif any(w in itext for w in ['back', 'spine']):
        return 'Back Injury'
    elif any(w in itext for w in ['oblique', 'abdominal', 'core']):
        return 'Core Injury'
    elif any(w in itext for w in ['covid', 'illness', 'flu', 'virus', 'cancer']):
        return 'Illness'
    elif any(w in itext for w in ['hamstring', 'quad', 'groin']):
        return 'Leg Muscle Injury'
    elif any(w in itext for w in ['knee', 'acl', 'mcl']):
        return 'Knee'
    elif any(w in itext for w in ['hand', 'wrist', 'finger', 'thumb']):
        return 'Hand/Wrist Injury'
    else:
        return 'Other'


# Apply our categoryize_injury function to raw injury text to make the
# ... new column with standardized categories we'll call `injury_group`
df['injury_group'] = df['injury / surgery'].apply(categorize_injury)

# Define the order we want for injury categories
injury_order = ['Elbow Injury', 'Shoulder Injury', 'Back Injury',
                'Core Injury', 'Illness', 'Leg Muscle Injury', 'Knee',
                'Hand/Wrist Injury', 'Other']

# Make labels cleaner for display by removing 'Injury' from categories
# Note: You can try it with the word 'Injury' at the end, but it gets pretty jumbled on the chart
clean_labels = [lbl.replace(" Injury", "") for lbl in injury_order]

# Sanity check: What are the Clean Labels in the dataframe going to look like on the chart?
print(clean_labels)



## Step 5: Static Visualizations with Seaborn - Boxplot and Violinplot
We are going to create two visualizations using Seaborn and Matplotlib

#### For this step we're going to:
1. *Create a Boxplot:* to visualize th edistribution of the 'age at injury' for each injury category
   - Boxplot will be split by position groups ('Pitcher' and 'Position Player') so we can compare their distributions side by side
   - Set order of the injury categories using our `injury_order` and then spoof up the x-axis labels using `FixedLocator` and `FixedFormatter`
2. *Create a Violin Plot:* to display the same data as the boxplot, but in a different style
   - Also split by position group, but shows quartile information inside the violins and offers a deeper view of distribution shape and density
  
We'll also do some **additional styling** by setting Seaborn's style to 'whitegrid' for a cleaner looking background, defining a custom color palette for the position groups for consistency and appeal, and creating custom legend handles using `Patch` from Matplotlilb to ensure clarity and effectiveness of our legend 


In [ ]:
##### Set Seaborn Style and Define Custom Color Palette
# First, we'll set seaborn's style to whitegrid for a clean-looking and modern background
# Documentation on Seaborn styling: http://seaborn.pydata.org/tutorial/aesthetics.html
sns.set(style='whitegrid')

# Define custom color palette for the two position groups
custom_palette = {
    'Pitcher': '#1F77B4',           # Yankees Navy
    'Position Player': '#FF7F0E'    # Just a nice orange I like
}


# --- Create the Boxplot ---
# Set figure size for boxplot
plt.figure(figsize=(12, 6))

# Visualize distribution of age at injury by injury category and position group
ax_box = sns.boxplot(
    data=df,                    # cleaned and merged dataframe
    x='injury_group',           # injury categories on x-axis
    y='age_at_injury',          # age at injury on y-axis
    hue='position_group',       # split data by position group
    order=injury_order,         # order of our categories we defined
    palette=custom_palette,     # our custom color palette we defined
    width=0.6,                  # adjust so it looks nice
    fliersize=0                 # remove outlier markers to declutter plot
)

# Set plot title and axis labels with additional spacing for readability
ax_box.set_title('Age at Injury by Category (Boxplot)', fontsize=16, pad=20)
ax_box.set_xlabel('Category', labelpad=15)
ax_box.set_ylabel('Age at Injury', labelpad=15)

# Use FixedLocator and FixedFormatter to make cleaner xticks and labels
# Produces consistent and appealing x-axis
ax_box.xaxis.set_major_locator(FixedLocator(range(len(injury_order))))
ax_box.xaxis.set_major_formatter(FixedFormatter(clean_labels))

# Custom legend handles using Patch for clarity
handles = [
    Patch(facecolor=custom_palette['Pitcher'], label='Pitcher'),
    Patch(facecolor=custom_palette['Position Player'], label='Position Player')
]

# Place legend outside main plot area so it doesn't overlap data
ax_box.legend(handles=handles, title='Position Group', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout to fit everything neatly and display boxplot
plt.tight_layout()
plt.show()


# --- Create the Violin Plot ---
# Set figure size for plot
plt.figure(figsize=(12, 6))

# Create violin plot to visualize same data, but with density and quartile ranges
ax_violin = sns.violinplot(
    data=df,                    # data source 
    x='injury_group',           # injury categories on x-axis
    y='age_at_injury',          # age at injury on y
    order=injury_order,         # same category order
    hue='position_group',       # split violin on position group
    palette=custom_palette,     # apply our color palette
    split=True,                 # so we can see both positon groups in one plot
    inner='quartile'            # display quartile info inside violin
)

# Set plot title and axis labels
ax_violin.set_title('Age at Injury by Category (Violin Plot)')
ax_violin.set_xlabel('Category', labelpad=15)
ax_violin.set_ylabel('Age at Injury', labelpad=15)

# Use FixedLocator and FixedFormatter to ensure x-axis ticks correctly labeled
ax_violin.xaxis.set_major_locator(FixedLocator(range(len(injury_order))))
ax_violin.xaxis.set_major_formatter(FixedFormatter(clean_labels))

# Custom legend for position groups added to the polot
ax_violin.legend(handles=handles, title='Position Group', bbox_to_anchor=(1.05, 1), loc='upper left')

# adjust layout and display the plot
plt.tight_layout()
plt.show()



## Step 6: Interactive Grouped Bar Chart with Plotly
First we are going to aggregate our injury data to derive statistics by year and injury category, then we will visualize these aggregate results in an interactive grouped bar chart using Plotly Express

#### For this step we're going to:
1. *Aggregate the Injury Data:* and then group by year and injury category
   - After we group by year and injury category, we will calculate:
     - The count of injuries using the count of `age_at_injury`
     - The median age at injury
     - The mean age at injury
   - We will produce a dataframe which we will call `subgroup_stats` that contains these statistics per group
2. *Calculate the Overall Counts:* per `injury_group` across all years and store it in `total_counts`
3. *Merge Aggregated Data:* to combine subgroup and overall data
   - Merge `subgroup_stats` and `total_counts` to get information in context along with the per-season stats.
   - Store the results in `bar_stats`
4. *Create Our Interactive Grouped Bar Chart:* and plot it with Plotly Express
    - Use `px.bar()` to create the grouped bar chart
      - x-axis is the year
      - y-axis displays the count of injuries
      - bars colored by injury group
    - Define the order of categories using the `injury_order` list from before
5. *Customize the Hover Information:* to desplay detailed information when we mouse over each bar
   - We will show the year, category, number of injuries, median age, and mean age
6. *Configure and Display the Chart:* by setting the x-axis ticks to show every season and then use `fig1.show()` to display the chart

In [ ]:
##### Calculate group injury data by year and inury category 
# Group the dataframe by 'year' and 'injury_group'
# For each group get the 'count', 'median_age' at injury, and 'mean_age' at injury
subgroup_stats = df.groupby(['year', 'injury_group']).agg(
    count=('age_at_injury', 'count'),
    median_age=('age_at_injury', 'median'),
    mean_age=('age_at_injury', 'mean')
).reset_index()           # ensures 'year' and 'injury_group' become columns again

# Calculate total injury counts per category, across all years
# Gives broader context for every injury category
# Renamed to `total_count`
total_counts = df.groupby('injury_group')['age_at_injury'].count().reset_index(name='total_count')

# Merge the subgroup statistics with overall counts
# The join here adds overall injury count for each group to the per-season stats
bar_stats = pd.merge(subgroup_stats, total_counts, on='injury_group', how='left')


# Create the interative grouped bar chart with Plotly Express
# Use `bar_stats` to build the chart
fig1 = px.bar(
    bar_stats,
    x='year',               # x-axis shows the year
    y='count',              # y-axis shows injury count for that year and category
    color='injury_group',   # color bars by injury group
    barmode='group',        # group the bars side by side
    title='Injuries by Year and Category',
    labels={'year': 'Year', 'count': 'Number of Injuries', 'injury_group': 'Category'},
    category_orders={'injury_group': injury_order},         # set injury category order
    custom_data=['injury_group', 'median_age', 'mean_age']  # attach extra data for hoverover feature
)

# Customize hover template to display the injury category and age stats
# Hover template uses Plotly's syntax to show the following:
# - year:             %{x}
# - injury category:  %{customdata[0]}
# - injury count:     %{y}
# - mean/median ages: customdata[1] and customdata[2]
# The <extra></extra> tags remove secondary box with trace information
fig1.update_traces(
    hovertemplate="Year: %{x}<br>Category: %{customdata[0]}<br>Injuries: %{y}<br>"
                  "Median Age: %{customdata[1]:.1f}<br>Mean Age: %{customdata[2]:.1f}"
                  "<extra></extra>"
)

# Configure x axis to display a tick for every year
# `dtick=1` makes sure each year appears as tick mark on x-axis
fig1.update_layout(xaxis=dict(dtick=1))

# Display interactive grouped bar chart
fig1.show()


## Step 7: Interactive Monthly Time Series Line Chart with Plotly
We are now goign to take our injury data and transform it to analyze monthly trends over the 2020-2024 seasons

#### For this step we're going to:
1. *Create a New Column for Month-Year:* by extracting the month and year from injury dates and then storing it as a string which makes it easier to group data monthly
2. *Aggregate Data by Month:* into a new `month_year` column to calculate the following:
   - With this aggregation, we now have the ability to view a monthly snapshot of injury statistics
3. *Convert Our New Month-Year to Datetime:* by converting the `month_year` back into a datetime object so we can plot our time series data properly
4. *Create Our Interactive Line Chart:* using Plotly Extress to visualize injuries per month over time
   - Customize hover template to allow users to see month, injury count, median age, and mean age on hover
   - Configure x-axis to display monthly ticks and keep the date range to only what we have in the data
   

In [ ]:
##### Create the new column getting the month and year from the injury date as a string
# Extract month and year from 'injury_date' column
# Using '.dt.to_period('M') converts datetime to a monthly Period, which we then convert to a string
# Documentation for .dt.to_period(): http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.to_period.html
df['month_year'] = df['injury_date'].dt.to_period('M').astype(str)

# Aggregate the counts, median ages, and mean ages of the injuries monthly
monthly_stats = df.groupby('month_year').agg(
    count=('age_at_injury', 'count'),
    median_age=('age_at_injury', 'median'),
    mean_age=('age_at_injury', 'mean')
).reset_index()              # makes sure 'month_year' goes back to regular column

# Convert 'month_year' string into a datetime object for correct plotting
# Documentation: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
monthly_stats['month_year_dt'] = pd.to_datetime(monthly_stats['month_year'], format='%Y-%m')

# Create the line chart using Plotly Express
# Plot the number of injuries over time with markers to indicate each point
# `px.line()` creates the interactive line chart
fig2 = px.line(
    monthly_stats,
    x='month_year_dt',                    # x-axis: datetime of month/year
    y='count',                           # y-axis: injuries count by month
    title='Injuries Over Time (Monthly)',
    labels={'month_year_dt': 'Month-Year', 'count': 'Number of Injuries'},
    markers=True,                    # show every data point with a marker
    custom_data=['median_age', 'mean_age']         # hover info uses these
)

# Customize hover template so user can see the month, injury count, and age stats
# Datetime format for hover template of %{x|%b '%y} makes datetime look something like: "Jan '22"
# The <extra></extra> tags keep extra information from appearing beyond our custom message
fig2.update_traces(
    hovertemplate="Month: %{x|%b '%y}<br>Injuries: %{y}<br>Median Age: %{customdata[0]:.1f}<br>Mean Age: %{customdata[1]:.1f}<extra></extra>"
)

# Configure x axis to display monthly ticks and set desired date range
fig2.update_layout(
    xaxis={
        "tickformat": "%b %y",                         # ticks formatted like "Jan 22", "Feb 22", etc.
        "tickmode": "linear",                                        # ensure ticks placed every month
        "dtick": "M1",                                                     # tick interval = one month
        "range": [pd.to_datetime("2020-01-01"), pd.to_datetime("2024-12-31")]  # set x-axis date range
    }
)

# Finally, display our interactive line chart!
fig2.show()


## Step 8: Bringing it All Together: Combine Visualizations
**Objective:**
1. Stack all charts vertically so we can compare them all at the same time
2. Display the two static Seaborn plots (the boxplot and violin plot) to visualize distribution of age at injury between position group and category of injury
3. Display the two interactive Plotly plots:
   - *Grouped Bar Chart*: visualizes injuries by year and injury category with additional, detailed information displayed upon hover
   - *Monthly Time Series Line Chart*: visualizes injury trends over time (monthly) with detailed hover over monthly data points as well

In [ ]:
##### Display All Charts Vertically
### Static Plots First (Seaborn and Matplotlib)
sns.set(style='whitegrid')

# Define custom color palette for our two specified position groups
custom_palette = {'Pitcher': '#1F77B4', 'Position Player': '#FF7F0E'}

# The Boxplot -------------------
plt.figure(figsize=(12, 6))
ax_box = sns.boxplot(
    data=df, x='injury_group', y='age_at_injury', hue='position_group',
    order=injury_order, palette=custom_palette, width=0.6, fliersize=0)

ax_box.set_title('Age at Injury by Category (Boxplot)', fontsize=16, pad=20)
ax_box.set_xlabel('Category', labelpad=15)
ax_box.set_ylabel('Age at Injry', labelpad=15)

# Set the x axis tick positions and format labels using FixedLocator and FixedFormatter
ax_box.xaxis.set_major_locator(FixedLocator(range(len(injury_order))))
ax_box.xaxis.set_major_formatter(FixedFormatter(clean_labels))

# Create custom legend handles to clearly delineate between position groups
handles = [Patch(facecolor=custom_palette['Pitcher'], label='Pitcher'),
           Patch(facecolor=custom_palette['Position Player'], label='Position Player')]
ax_box.legend(handles=handles, title='Position Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# The Violinplot -----------------------
plt.figure(figsize=(12, 6))
ax_violin = sns.violinplot(
    data=df, x='injury_group', y='age_at_injury', order=injury_order, hue='position_group',
    palette=custom_palette, split=True, inner='quartile')

ax_violin.set_title('Age at Injury by Category (Violin Plot)', fontsize=16, pad=20)
ax_violin.set_xlabel('Category', labelpad=15)
ax_violin.set_ylabel('Age at Injury', labelpad=15)

# Format x axis ticks
ax_violin.xaxis.set_major_locator(FixedLocator(range(len(injury_order))))
ax_violin.xaxis.set_major_formatter(FixedFormatter(clean_labels))
ax_violin.legend(handles=handles, title='Position Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### Interactive Plots Next
# Grouped Bar Chart
subgroup_stats = df.groupby(['year', 'injury_group']).agg(
    count=('age_at_injury', 'count'),
    median_age=('age_at_injury', 'median'),
    mean_age=('age_at_injury', 'mean')
).reset_index()

# Compute overall injury counters per category per year
total_counts = df.groupby('injury_group')['age_at_injury'].count().reset_index(name='total_count')
# Merge subgroup stats with overall counts
bar_stats = pd.merge(
    subgroup_stats, total_counts, on='injury_group', how='left')

# Create the bar chart with Plotly Express
fig_bar = px.bar(
    bar_stats, x='year', y='count', color='injury_group',
    barmode='group', title='Injuries by Year and Category',
    labels={'year': 'Year', 'count': 'Number of Injuries',
            'injury_group': 'Category'},
    category_orders={'injury_group': injury_order},
    custom_data=['injury_group', 'median_age', 'mean_age']
)

# Update Hover template
fig_bar.update_traces(
    hovertemplate="Year: %{x}<br>Category: %{customdata[0]}<br>Injuries: %{y}<br>Median Age: %{customdata[1]:.1f}<br>Mean Age: %{customdata[2]:.1f}"
)

# Force x axis to have a tick every year
fig_bar.update_layout(xaxis=dict(dtick=1))
fig_bar.show()

# Monthly Time Series Line Chart
# Create column showing month/year from injury date
df['month_year'] = df['injury_date'].dt.to_period('M').astype(str)

# Aggregate injuries by month: get the count, median age and mean age
monthly_stats = df.groupby('month_year').agg(
    count=('age_at_injury', 'count'),
    median_age=('age_at_injury', 'median'),
    mean_age=('age_at_injury', 'mean')
).reset_index()

# Convert month/year strings into datetime objects
monthly_stats['month_year_dt'] = pd.to_datetime(monthly_stats['month_year'], format='%Y-%m')

# Create the interactive line chart with Plotly Express
fig_line = px.line(
    monthly_stats,
    x='month_year_dt',
    y='count',
    title='Injuries Over Time (Monthly)',
    labels={'month_year_dt': 'Month-Year', 'count': 'Number of Injuries'},
    markers=True,
    custom_data=['median_age', 'mean_age']
)

# Customize hover template to show month, injury count, and age statistics
fig_line.update_traces(
    hovertemplate="Month: %{x|%b '%y}<br>Injuries: %{y}<br>Median Age: %{customdata[0]:.1f}<br>Mean Age: %{customdata[1]:.1f}"
)

# Configure x axis to set tick format, mode, and range
fig_line.update_layout(
    xaxis={
        "tickformat": "%b '%y",
        "tickmode": "linear",
        "dtick": "M1",
        "range": [pd.to_datetime("2020-01-01"), pd.to_datetime("2024-12-31")]
    }
)

fig_line.show()


## Conclusion: MLB Injury Analysis by Age and Position Group

#### Key Takeaways:
1. *Distribution of Injury Ages:*
   - The **boxplot** and **violin plot** visualilzations provide us a clear view of how player ages at time of injury vary across different injury categories and between the position groups we defined (pitchers vs position players)
   - These plots highlighted the medians and the spread of ages for the discrete injury categories
2. *Yearly Injury Trends:*
   - The **grouped bar chart** allowed us to dig into the weeds a bit by breaking down the injury counts yearly and by injury category
   - Also included detailed statistical insights such as median age and mean age at injury for each category
3. *Monthly Injury Trends:*
   - The **monthly time series line chart** showed how injury events have evolved over time on a month-to-month basis
   - We can also gain insight into seasonal patterns and shifts in injury frequencies by visualizing monthly trends

# MLB Analytics Dashboard: WAR vs Salary with Spotlight and Interactive Filters

#### Objective: Advanced Plot
**What We will do in this section:**
1. *Integrate* multiple data sources (batters, pitchers, people, WAR, salary) to explore the interplay between salary and performance
2. *Compare* team performance and league-wide trends to highlight differences in efficiency and identify potential contract sinkholes
3. *Filter* stats-based insights by running the analysis for all positions, for pitchers, or for position players
4. *Enable* seasonal measurement granularity by allowing the user to analyze statistics for individual seasons or the combined statistics over all seasons
5. *Spotlight* standout or underperforming players based on salary values relative to WAR
6. *Incorporate* an interactive legend and filter controls which allow age-groupings to be filtered on or off for the user

**What are we going to try to find out?**
1. How to teams and the league overall compare when converting salary into player performance?
   - Can we identify efficiency benchmarks or warning signs between varying roles and ages leading to great deal or a multi-million dollar contract sinkhole?
2. How does player age influence the dynamics of salary and performance?
   - How might we identify trends in longevity and performance and pivot team strategy for roster makeup?
  
#### Data Sources:
1. `people.csv`
2. `stats_batters_all.csv`
3. `jeffbagwell_war_historical_2023.csv`
4. `mlb_salary_data.csv`

## Import the Required Libraries

#### We must import the following libraries in order to do our analysis:
Though we've done some of these above, we'll put in all the ones we need to use here for instructive purposes, even if it's a repeat import

The following **libraries** are necessary to import for our analysis
- **Pandas and Numpy:** for data manipulation
- **Seaborn and Matplotlib:** for building static visualizations
- **Plotly:** for building interactive visualizations (Graph Objects and Express)
- **ipywidgets and IPython.display:** for creation and display of our UI components
- **re, FixedLocator, FixedFormatter, and Patch:** for data cleaning and plot fine-tuning

In [ ]:
import pandas as pd     # Documentation: http://pandas.pydata.org/docs/
import numpy as np      # Documentation: http://numpy.org/doc/
import seaborn as sns           # Documentation: http://seaborn.pydata.org/
import matplotlib.pyplot as plt           # Documentation: http://matplotlib.org/stable/contents.html
import plotly.express as px             # Documentation: http://plotly.com/python/plotly-express/
import plotly.graph_objects as go        # Documentation: http://plotly.com/python/reference/
import re                 # Documentation: http://docs.python.org/3/library/re.html
import ipywidgets as widgets       # Documentation: http://ipywidgets.readthedocs.io/
from ipywidgets import VBox, HBox, Output, Layout, HTML
from IPython.display import display, clear_output
from matplotlib.patches import Patch
from matplotlib.ticker import FixedLocator, FixedFormatter
from plotly.subplots import make_subplots

## Step 1: Loading of Data and Initial Data Prep

**In this step we're going to:**
1. Load four datasets using the Pandas `read_csv()` function
2. Standardize column names and player names for consistency across dataframes
3. Merge datasets on the common keys: `player_name` and `year` to make a unified dataset called 'merged'
4. Calculate player ages and classify them by position group
5. Filter out non-positive salaries, missing WAR values, and duplicate records

In [ ]:
##### Load the datasets using read_csv()
# Load the stats, salary data, WAR metrics, and player demographics
bat_df = pd.read_csv('assets/stats_batters_all.csv', encoding='latin1')
sal_df = pd.read_csv('assets/mlb_salary_data.csv', encoding='latin1')
war_df = pd.read_csv('assets/jeffbagwell_war_historical_2023.csv', encoding='latin1')
people_df = pd.read_csv('assets/people.csv', encoding='latin1')

# Standardize the df column names by stripping extra spaces and converting to lowercase
# This will make merging and referencing columns easier downstream
for df in [bat_df, sal_df, war_df, people_df]:
    df.columns = df.columns.str.strip().str.lower()   # http://pandas.pydata.org/docs/reference/api/pandas.Series.str.html

# Standardize player names by removing extra spaces, coverting to lower, and removing periods
if 'name' in bat_df.columns:
    bat_df['player_name'] = bat_df['name'].str.strip().str.lower().str.replace('.', '', regex=False)
if 'name' in sal_df.columns:
    sal_df['player_name'] = sal_df['name'].str.strip().str.lower().str.replace('.', '', regex=False)
if 'player_name' in war_df.columns:
    war_df['player_name'] = war_df['player_name'].str.strip().str.lower().str.replace('.', '', regex=False)
if 'namefirst' in people_df.columns and 'namelast' in people_df.columns:
    people_df['player_name'] = (people_df['namefirst'].str.strip() + ' ' + people_df['namelast'].str.strip()
                                ).str.lower().str.replace('.', '', regex=False)
else:
    raise KeyError('Missing first/last name fields in people.csv')

# Rename columns for consistency
# Convert 'season' to 'year' for bat_df and sal_df, and 'year_id' to 'year' for war_df
bat_df = bat_df.rename(columns={'season': 'year'})
sal_df = sal_df.rename(columns={'season': 'year'})
war_df = war_df.rename(columns={'year_id': 'year'})

#Sanity check: what do our dataframes look like? Uncomment the lines one at a time below to check:
# bat_df
# sal_df
# war_df

# Merge datasets on 'player_name' and 'year' using inner join to keep only matching records
# First, we're going to merge WAR and salary data, then we're going to merge that
# --- with the people data and keep only what we need
merged = pd.merge(war_df, sal_df, on=['player_name', 'year'], how='inner')
merged = pd.merge(merged, people_df[['player_name', 'birthyear']], on='player_name', how='left')

# Next create an age column by subtracting birth year from current season-year
# While we're here, determine the position group the player belongs to based on the 'is_p' flag
merged['age'] = merged['year'] - merged['birthyear']
merged['position_group'] = np.where(merged['is_p'] == "Y", "Pitcher", "Position Player")

# Assign WAR based on position: pwar162 for pitchers and bwar162 for position players
merged['war'] = np.where(merged['is_p'] == "Y", merged['pwar162'], merged['bwar162'])
salary_col = 'salary_x' if 'salary_x' in merged.columns else 'salary'

# Drop any records missing salary or WAR vals and filter out non-positive salary vals
merged = merged.dropna(subset=[salary_col, 'war'])
merged = merged[merged[salary_col] > 0]

# Get rid of duplicate entries for each player based on salary and year but keep the first occurence
merged = merged.sort_values('year').drop_duplicates(subset=['player_name', salary_col], keep='first')

# Sanity check: get a look at what's in the merged dataframe after all this manipulation and merging
merged




## Step 2: Calculate Our Derived Metrics and Our Age Categorization Method

**In this step we're going to:**
1. Enrich our data by supplementing with additional performance metrics
2. Compute a value score that juxtaposes WAR against salary (which we'll normalize in millions) to give us a cost/performance metric
3. Calculate our `sinkhole_metric` which highlights players with low WAR relative to their annual salary that year
4. Define and apply a `categorize_age` function to place players into age brackets (under-25, 25-29, 30-34, and over-35)

With these additions to our project, we enable increased context and depth to be cleaned from our visualization 

In [ ]:
##### Calculate Our Own Performance Metrics
# Make our 'value_score' by dividing WAR by salary in millions + 1
# The idea here is to quantify the performance of any player relative to their salary
merged['value_score'] = merged['war'] / ((merged[salary_col] / 1_000_000) + 1)

# Calculate my favorite named-metric....the 'sinkhole_metric' for players with WAR under 1
# This spotlights players who are getting paid way too much to be playing so poorly
merged['sinkhole_metric'] = np.where(merged['war'] < 1,
                                    (1 - merged['war']) * merged[salary_col],
                                    np.nan)

# Create a function to categorize player age into age brackets that make sense
def categorize_age(age):
    if age < 25:
        return 'Under 25'
    elif age < 30:
        return '25-29'
    elif age < 35:
        return '30-34'
    else:
        return '35+'


# Apply 'categorize_age' function to the 'age' column
merged['age_bracket'] = merged['age'].apply(categorize_age)

# Sanity check: take a look at what our data looks like
# - Notice anything different?
merged

# We have the same number of rows, but we should have three additional columns...
# .....the ones we just created
# - 'value_score'
# - 'sinkhole_metric'
# - 'age_bracket'


## Step 3: Mapping Our Age Bracket Colors and Creating an Age Correction Function

**In this step we're going to:**
1. Set a color mapping for our age brackets so we can readily distinguish age groups between each other in the chart
2. Define the `get_corrected_age` helper function to make sure player ages are displayed correctly in our hover-overs
   - Handles edge cases where a player's age might be ridiculously high by calculating a median valid age based on historical data


In [ ]:
##### Define the Color Mapping
age_colors = ['#66C2A5', '#FC8D62', '#8DA0CB', '#E78AC3']
age_map = {
    'Under 25': age_colors[0],
    '25-29': age_colors[1],
    '30-34': age_colors[2],
    '35+': age_colors[3]
}


# Create Our Utility Function
def get_corrected_age(row):
    if row['age'] <= 45:
        return f"Age: {int(row['age'])}"
    grp = merged[merged['player_name'] == row['player_name']]
    valid_ages = grp[grp['age'] <= 45]['age']
    if not valid_ages.empty:
        return f"Age: {int(valid_ages.median())}"
    return f"Age: Unknown<br>{int(row['year'])} Season"



## Step 4: Setting Up the Constants

**In this step we're going to:**
Set dimensions and thresholds for filtering and styling which we will use downstream in the chart

In [ ]:
##### Define our constants for layout dimensions and marker styles
CHART_WIDTH = 800     # width in pixels
CHART_HEIGHT = 600    # height in pixels
RIGHT_WIDTH = 400     # width for table side container

# Threshold constants for filtering the spotlight player data
TOP_MIN_SALARY = 0.76e6         # $750k
TOP_MAX_SALARY = 12.49e6        # $12.49M
EXTENDED_MAX_SALARY = 20e6      # $20M
INITIAL_SINKHOLE_MIN = 15e6     # $15M

# Marker size and border constants
NON_SPOT_SIZE = 8
SPOT_SIZE = 24
NON_SPOT_BORDER = 1
SPOT_LINE_WIDTH = 2.5


## Step 5: Define Our Plotting Function

**In this step we're going to:**

Create the `plot_plotly_war_vs_salary()` function which will build the interactive scatter plot chart of WAR vs Salary, a player spotlight table, and an interactive legend. It will use various filtering parameters like team, season, spotlight type, and player role. Our function will do the following:
1. **Data Filtering:**
   - Create copy of global dataframe 'merged' which contained the player data
   - Filter data based on provided parameters:
     - *Season:* either an individual season or 'All Years'
     - *Team:* either an individual team of 'All Teams'
     - *Role:* either 'Pitcher', or 'Position Player', or 'All Roles'
2. **Spotlight Filtering:**
   - Create a new column called `spotlight` to make certain players for highlighting in the chart
   - Depending on which kind of spotlight, apply a filtering strategy:
     - *Top WAR-Per-Dollar:* selects the best candidates based on WAR and salary thresholds; can be done for individual team or 'All Teams'
     - *Worst Value Sinkholes:* iteratively lowers the threshold until enough candidates are found, then selects those players
   - Mark the selected candidates as spotlight players
3. **Data Prep for Plotting:**
   - Format the saslary values and round them
   - Create hover text for each player with details like name, year, WAR, salary, team, and role
   - Define groupoing labels and assign marker symbols and sizes based on player attributes
4. **Building the Plotly Figure:**
   - Group data by age bracket and position group to create separate scatter traces
   - Configure the marker properties for each trace (color, size, symbol, border)
   - Create and configure Plotly figure (titles, axis labels, legend, layout settings)
5. **Create the Interactive Plot:**
   - Build HTML table displaying spotlight players
   - Create interactive legend using ipywidgets toggle buttons that let you show or hide the traces on the chart
   - Combine Plotly chart and spotlight table with legend into one layout
    

#### That's a lot!!! Let's get coding!


In [ ]:
##### This will be our main function
def plot_plotly_war_vs_salary(team, season, spotlight, role):
    # --- Data Filtering Section ---
    # Copy global merged df to keep from changing original data
    # This is a standard safety measure when processing data
    df = merged.copy()

    # Apply the season filter: we will use a bool mask to filter the rows
    # - If 'all years' isn't selected, select rows for the selected year
    if season !='All Years':
        df = df[df['year'] == int(season)]
    # Apply team filter: if not 'all teams' select rows for selected team
    if team != 'All Teams':
        df = df[df['team'] == team]
    # Apply role filter: if not 'all roles' select rows for selected player role
    if role != 'All Roles':
        df = df[df['position_group'] == role]

    # --- Spotlight Filtering Section ---
    # We're now ready to create our new 'spotlight' column in df and set rows
    # initially to False because by default no player is highlighted
    df['spotlight'] = False
    spotlight_df = pd.DataFrame()        # Creates empty df for spotlight players

    # Determine number of spotlight entries to show based on team filter
    # if all teams is selected -> 5 spotlight players; if any specific team is selected -> 3 spotlight players
    spotlight_n = 5 if team == 'All Teams' else 3

    # Filter first for top war per dollar
    # What we're trying to do is find and highlight players who have positive WAR
    # ....and also salaries within defined thresholds to enforce the value proposition
    # We are going to grab multiple candidate selections for robustness
    if spotlight == 'Top WAR-Per-Dollar':
        # For a specific team
        if team != 'All Teams':
            # Select candidates with positive WAR and salary within defined thresholds
            cand1 = df[(df['war'] > 0) & (df[salary_col] >= TOP_MIN_SALARY) & (df[salary_col] <= TOP_MAX_SALARY)]
            # Sort candidates by highest WAR first and if there are equal WARs the lower salary wins
            cand1 = cand1.sort_values(['war', salary_col], ascending=[False, True])
            # Include players with very high WAR...like a WAR > 8 regardless of salary
            war8plus = df[df['war'] > 8]
            # Combine candidate sets and remove duplicates to remove redundancy
            combined = pd.concat([cand1, war8plus]).drop_duplicates()

            # If less than 3 candidates found, add candidates from extended salary ranges
            if len(combined) < 3:
                # WAR > 8 with salary moved to extended range 'TOP_MAX_SALARY' -> 'EXTENDED_MAX_SALARY'
                cand2 = df[(df['war'] > 8) & (df[salary_col] > TOP_MAX_SALARY) & (df[salary_col] <= EXTENDED_MAX_SALARY)]
                cand2 = cand2.sort_values(['war', salary_col], ascending=(False, True))
                combined = pd.concat([combined, cand2]).drop_duplicates()
            if len(combined) < 3:
                # players with positive WAR but salary below threshold
                cand3 = df[(df['war'] > 0) & (df[salary_col] < TOP_MIN_SALARY)]
                cand3 = cand3.sort_values(['war', salary_col], ascending=[False, True])
                combined = pd.concat([combined, cand3]).drop_duplicates()

            # Sort and select top 3 candidates based on WAR and salary
            combined = combined.sort_values(['war', salary_col], ascending=[False, True]).head(3)
            spotlight_df = combined
        else:
            # For 'all teams' apply the broader filter and select top 5
            quad_df = df[(df['war'] > 0) & (df[salary_col] >= TOP_MIN_SALARY) & (df[salary_col] <= TOP_MAX_SALARY)]
            quad_df = quad_df.sort_values(['war', salary_col], ascending=[False, True])
            spotlight_df = quad_df.head(5)
        
        # Mark spotlight rows in dataframe
        df.loc[spotlight_df.index, 'spotlight'] = True

    # Now filter for worst contracts (low WAR and high salaries)
    # We're going to start with an initial salary limit, but if we don't have enough candidates we will 
    # ...lower the limit by $1M until we get enough
    elif spotlight == 'Worst Value Sinkholes':
        limit = INITIAL_SINKHOLE_MIN
        cand = pd.DataFrame()
        while limit >= 0:
            # Filter for players with WAR < 1 and salary above current limit
            cand = df[(df['war'] < 1) & (df[salary_col] >= limit)].copy()
            # if we have enough players or if the limit is down to 0, break out of here
            if len(cand) >= spotlight_n or limit <= 0:
                break
            limit -= 1e6   # Lower salary threshold by $1M
        if not cand.empty:
            # Sort candidates to find the lowest WAR and highest salary
            cand_br = cand.sort_values(['war', salary_col], ascending=[True, False])
            most_bottom_right = cand_br.iloc[0:1]
            # Fill missing values for sinkhole metric and sort
            cand['sinkhole_metric'] = cand['sinkhole_metric'].fillna(0)
            cand_top = cand.sort_values('sinkhole_metric', ascending=False)
            combined = pd.concat([most_bottom_right, cand_top]).drop_duplicates()
            spotlight_df = combined.head(spotlight_n)
            # These players are going to be marked in the df
            df.loc[spotlight_df.index, 'spotlight'] = True

    # --- Data Prep ---
    # Convert salary to millions and round the value
    df['Salary_M'] = (df[salary_col] / 1_000_000).round(2)

    # Make hover text for tooltips with player details in Plotly
    # We will create a string including Player Name, Year, WAR, Salary in millions, Team, and Role
    df['hover'] = (df['player_name'].str.title() +
                   "<br>Year: " + df["year"].astype(str) +
                   "<br>WAR: " + df["war"].round(2).astype(str) +
                   "<br>Salary: $" + df["Salary_M"].astype(str) +
                   "<br>Team: " + df["team"].fillna("N/A") +
                   "<br>Role: " + df["position_group"].fillna("N/A"))

    # Create group labels by combining age bracket and position group
    df['group'] = df['age_bracket'] + ', ' + df['position_group']
    # Assign marker symbols: pitchers -> circle, position players -> diamond
    # For documentation on Plotly's marker symbols: http://plotly.com/python/marker-style/
    df['group_symbol'] = df['position_group'].apply(lambda x: 'circle' if x == 'Pitcher' else 'diamond')
    # Make marker size dependent on spotlight status
    # If player is a spotlight player use the large marker, otherwise use the small marker 
    df['size_col'] = np.where(df['spotlight'], SPOT_SIZE, NON_SPOT_SIZE)

    # For Logic in the styling, set a sort order for age groups
    # Sort groups from youngest to oldest for legend ordering
    age_order = {'Under 25': 0, '25-29': 1, '30-34': 2, '35+': 3}

    # Group and sort df by age bracket and position group
    groups = sorted(list(df.groupby(['age_bracket', 'position_group'])), key=lambda g: age_order.get(g[0][0], 99))

    # Sort groups using age_order mapping so under 25 comes first
    # groups = sorted()

    # Define sort key that sorts by age bracket order
    def sort_key(item):
        age_bracket, _ = item[0]
        # Default to high val if not found
        return age_order.get(age_bracket, 99)


    # --- Build Plotly Traces and Start Configuring the Fig
    # Create initially empty list to hold scatter traces
    traces = []
    # Group df by 'age bracket' and 'position group'
    # Each group gets its own trace for customization of colors and symbols
    for (ab, pg), sub_df in groups:
        if sub_df.empty:
            continue     # Skip this group if no data exists
        # Get color from earlier mapping based on 'age bracket'
        color_val = age_map[ab]
        # Choose symbol based on position group
        symbol_val = 'circle' if pg == 'Pitcher' else 'diamond'

        # Now we're goign to create our scatter traces
        traces.append(
            go.Scatter(
                x=sub_df[salary_col],     # x axis vals = salary
                y=sub_df['war'],          # y axis vals = WAR
                mode='markers',           # display as points
                name=f'{ab}, {pg}',       # trace name for the legend
                showlegend=True,
                marker=dict(
                    size=sub_df['size_col'],        # marker size
                    color=color_val,                # marker color from mapping
                    symbol=symbol_val,              # marker symbol by position group
                    line=dict(
                        # Thicker border for spotlight traces
                        width=[SPOT_LINE_WIDTH if s == SPOT_SIZE else NON_SPOT_BORDER for s in sub_df['size_col']],
                        color='black'
                    ),
                    opacity=0.9           # A little bit of opacity change for presentation
                ),
                text=sub_df['hover'],     # Our tooltip interactivity text
                hoverinfo='text'
            )
        )

    # Create Plotly figure with our now-defined traces
    fig = go.Figure(data=traces)

    # Update overall look and feel of figure's layout: title, axis labels, margins, legend
    fig.update_layout(
        title=f'WAR vs Salary - {spotlight}',          # Dynamic title with spotlight criteria 
        xaxis_title='Salary (USD)',
        yaxis_title='Wins Above Replacement (WAR)',
        hovermode='closest',
        width=CHART_WIDTH,
        height=CHART_HEIGHT,
        margin=dict(l=30, r=30, t=50, b=50),
        legend=dict(
            orientation='v',
            x=1.02,
            y=1,
            xanchor='left',
            yanchor='top',
            borderwidth=1,
            bordercolor='black',
            itemwidth=90,
            title='Age and Position'
        )
    )

    # Configure x axis and y axis properties
    # Prefix x axis vals with '$'
    # Enforce x axis begins at 0
    # Add zero line on y axis
    fig.update_xaxes(type='linear', tickprefix='$', rangemode='tozero')
    fig.update_yaxes(zeroline=True)


    # --- Build the Spotlight Table Using HTML ---
    # I am a frontend software engineer by day, so I thought we would have some
    # ....fun building out some HTML for our project as well
    # Admittedly, in a software engineering project we'd have styling in a separate file,
    # ....but for convenience and expediency, we'll do it here

    # Initialize 'table_widget' with default value in case no spotlight players exist
    table_widget = HTML(value="<div style='font-family: Arial, sans-serif;'><em>No spotlight players found.</em></div>")

    # Begin building HTML string for table
    table_html = f"""
    <div style="font-family: Arial, sans-serif;
                border: 1px solid #AAA;
                padding: 5px;
                width: {RIGHT_WIDTH}px;
                height: {CHART_HEIGHT}px;
                box-sizing: border-box;
                overflow: hidden;">
    <h3 style="margin: 0; font-size: 14px; text-align: center;">Spotlight Players</h3>
    <table style="border-collapse: collapse; width: 100%; font-size: 10px;">
        <thead>
            <tr style="background-color: #F2F2F2;">
                <th style="border: 1px solid #DDD; padding: 4px;">Player</th>
                <th style="border: 1px solid #DDD; padding: 4px;">WAR</th>
                <th style="border: 1px solid #DDD; padding: 4px;">Salary</th>
                <th style="border: 1px solid #DDD; padding: 4px;">Age/Season</th>
                <th style="border: 1px solid #DDD; padding: 4px;">Team</th>
                <th style="border: 1px solid #DDD; padding: 4px;">Role</th>
            </tr>
        </thead>
        <tbody>
    """
    spotlight_only = df[df['spotlight']]
    if not spotlight_only.empty:
        for _, row in spotlight_only.iterrows():
            # Using 'get_corrected_age()' helper function from earlier here to adjust age/season info
            corrected_age = get_corrected_age(row)
            table_html += f"<tr>"
            table_html += f"<td style='border:1px solid #DDD; padding:4px;'>{row['player_name'].title()}</td>"
            table_html += f"<td style='border:1px solid #DDD; padding:4px;'>{row['war']:.2f}</td>"
            table_html += f"<td style='border:1px solid #DDD; padding:4px;'>${(row[salary_col]/1_000_000):.2f}M</td>"
            table_html += f"<td style='border:1px solid #DDD; padding:4px;'>{corrected_age}</td>"
            table_html += f"<td style='border:1px solid #DDD; padding:4px;'>{row['team']}</td>"
            table_html += f"<td style='border:1px solid #DDD; padding:4px;'>{row['position_group']}</td>"
            table_html += "</tr>"
        table_html += "</tbody></table></div>"
        table_widget = HTML(value=table_html)

    # --- Build Legend and Combine Layout
    # Make empty 'legend items' list to keep toggle buttons for each trace
    legend_items = []

    # Make an output widget to display chart
    chart_container = Output(layout=Layout(width=f'{CHART_WIDTH}px', height=f'{CHART_HEIGHT}px'))
    with chart_container:
        # render the figure to the screen
        fig.show()

    # Now we build the 'right container'to hole spotlight players and legend
    right_container = VBox(
        [table_widget],
        layout=Layout(width=f'{RIGHT_WIDTH}px', height=f'{CHART_HEIGHT}px',
                      border='1px solid #AAA', padding='5px',
                      box_sizing='border-box', overflow='hidden')
    )

    # Combine the chart and right container horizontally in an HBox
    main_container = HBox(
        [chart_container, right_container],
        layout=Layout(width=f'calc({CHART_WIDTH}px + {RIGHT_WIDTH}px + 20px)', height=f'{CHART_HEIGHT}px')
    )

    # Finally, return the complete chart as an ipywidgets HBox
    return main_container


### Wow, that was a lot! But you made it!

**But what did all that code just do?**
- Defined a complete, interactive dashboard component that can be called to display the chart in this notebook

## Step 6: Set Up Our Widgets and Plotting Update Function

**In this step we're going to:**
Next up, we're going to make interactive filter widgets with ipywidgets functionality and control of our chart. They will let the user select between various teams, roles, seasons, and whether or not to view spotlight players to dynamically update the plot. We've already created the `plot_plotly_war_vs_salary()` function to build it for us

1. **Create Dropdown Widgets:**
   - *Team Dropdown:* Shows a selection of "All Teams" and a sorted list of teams from the global `merged` dataframe
   - *Role Dropdown:* Displays options for "All Roles" and either "Pitcher" or "Position Player"
   - *Season Dropdown:* Offers the user to select "All Years" or one of a sorted list of available seasons played
   - *Spotlight Dropdown:* Lets the user choose to view "All Players" or highlight the spotlight players depending on their selection of "Top WAR-Per-Dollar" and "Worst Value Sinkholes"
2. **Set Up Dynamic Output Widget:**
   - Holds the dashboard so that whenever a widget value changes, we can update the chart
3. **Define Our Update Function:**
   - This new function `update_plot(change=None)` wipes the current output and calls `plot_plotly_war_vs_salary()` with current widget values
   - We then attach this function to each widget so our chart is updated whenever a user changes a filter value
4. **Assemble and Display:** (we finally made it!)
   - Create a container using HBox for filter widgets and display the filter widgets and ***our dynamic dashboard***

In [ ]:
##### Create the Interactive filter widgets using ipywidgets
# Team Dropdown
team_dropdown = widgets.Dropdown(
    # Get 'All Teams' in dropdown first, then the sorted teams from dataframe
    options=['All Teams'] + sorted(merged['team'].dropna().unique()),
    value='All Teams',   # set 'All Teams' as default selection of dropdown
    description='Team:'  # sets label displayed next to the dropdown to 'Team:'
)

# Role Dropdown
role_dropdown = widgets.Dropdown(
    options=['All Roles', 'Pitcher', 'Position Player'],
    value='All Roles',    # Defaults to 'All Roles' so there is no initial filtering
    description='Role:'   # Labels the dropdown for clarity
)

# Season Dropdown: Provides available years from df in addition to the 'All Years' option
# Extract years from 'merged' and convert to strings
years = sorted(merged['year'].dropna().unique().astype(str))
season_dropdown = widgets.Dropdown(
    # Get 'All Years' first then sorted years from dataframe
    options=['All Years'] + years,
    value='All Years',             # Defaults selection to 'All Years'
    description='Season:'          # Dropdown label
)

# Spotlight Dropdown: User chooses which, if any, filter to spotlight players with
spotlight_dropdown = widgets.Dropdown(
    options=['All Players', 'Top WAR-Per-Dollar', 'Worst Value Sinkholes'],
    value='All Players',         # Default to 'all players'
    description='Spotlight:'     # Dropdown label
)

# Set Up an Output widget to hold dynamic chart
# displays updated chart every time a filter is applied or changed
dynamic_output = Output()


# Define function to update chart when a widget value changes
def update_plot(change=None):
    with dynamic_output:
        # Clear current output for smooth UX purposes
        dynamic_output.clear_output(wait=True)
        # Generate updated chart using current vals from each widget
        container = plot_plotly_war_vs_salary(
            team=team_dropdown.value,          # Current team selection
            season=season_dropdown.value,       # Current season selection
            spotlight=spotlight_dropdown.value,   # Current spotlight selection
            role=role_dropdown.value           # Current role selection
        )
        # Display new, updated chart with the Output widget
        display(container)

# Attach 'update_plot' function as observer to each widget
# - Ensures chart updates automagically whenever we change a filter
for dropdown in [team_dropdown, role_dropdown, season_dropdown, spotlight_dropdown]:
    # trigger 'update_plot()' when widget's value changes
    dropdown.observe(update_plot, names='value')

# Initial call to display the chart with default filters
update_plot()


# Create container for the filter widgets and display the beast!
filter_container = HBox(
    # All dropdowns arranged in horizontal row
    [team_dropdown, role_dropdown, season_dropdown, spotlight_dropdown],
    layout=Layout(width='100%', height='80px', align_items='center', padding='5px', overflow_x='hidden')
)

# Finally, display filter container and dynamic chart output together
display(filter_container, dynamic_output)


## Conclusion: MLB WAR vs Salary with Spotlight and Interactive Filters
**But what did we just do?**
- What we just did was create an interactive dashboard to examine the relationship between MLB player salaries and their performance, measured by WAR
  1. Loaded and preprocessed multiple datasets with Pandas
  2. Calculated new metrics for analyzing player efficiency and underformance (`value_score` and `sinkhole_metric`)
  3. Grouped players into age brackets and mapped their colors for clear visualizations
  4. Created an interactive Plotly scatter plot with detailed tooltips and a bespoke HTML table for our spotlight players just for fun
  5. Used our ipywidgets to enable dynamic filtering and real-time chart updates within the dashboard

# Conclusion: Insights and Future Directions in MLB Analytics
As we wrap up this dashboard project, before we look ahead, lets take a quick look back and what we've accomplished here. 

#### Key Insights:
1. **Comprehensive Data Integration:**
   - We merge batting, pitching, salary, WAR, and demographic data to create a unified dataset that offers a complete picture of MLB player performance and financial dynamics
   - This integration allowed us to correlate player performance with salary and health measurements
2. **WAR vs Salary:**
   - Our interactive chart revealed serious trends in how teams successfully or unsuccessfully invest in talent
   - This study highlighted standout, or "spotlight", players and potential contract inefficiencies (sinkholes) that could tank team strategy
3. **Injury Patterns by Age and Position:**
   - Visualizations like boxplots, violin plots, and grouped bar charts illustrated clear differences between pitcher and position player injury occurrences
   - Identified specific age ranges where injuries are more common, offering insight into team makeup vulnerabilities and longevity trends
4. **Temporal Trends in Injury Data:**
   - Also in the injury patterns chart, the monthly time series analysis shows seasonal patterns and gradual shifts in injury frequency
   - Give temporal context to injury data, allowing for planning for future seasons

#### Future Directions:
1. **Deeper Analysis:**
   - We could include more granular performance metrics, a deeper dive into specific injury types and their long-term effects, or even ***create*** new metrics
   - We could use predictive modeling to attempt to forecast injury risks or performance trends based on historical data
   - We could see how age and injuries play into contract valuation as far as extreme value or extreme sinkhole status
2. **Enhanced Interactivity:**
   - Future iterations can incorporate additional filters, more sophisticated user controls, or even real-time data updates
   - Technology advancements could provide further insights and usability of the chart which we don't have availability at this moment
3. **Broader Applications:**
   - Other sports, business contract/performance valuations, health and fitness domains


